# 主要目标：跑通 Baseline

## 1 环境配置

我本地已经有这些包了，故省略

## 2 导包

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LinearRegression

## 3 数据预处理

In [3]:
base_dir = Path("./data") # 数据根目录

# 读取数据
electricity_price = pd.read_csv(base_dir / "electricity price.csv")
unit = pd.read_csv(base_dir / "unit.csv")

我们先来看看数据里有什么：

In [4]:
electricity_price.head()

,day,time,demand,clearing price (CNY/MWh)
0,2021/12/1,0:15,40334.18,350.80
1,2021/12/1,0:30,40523.15,350.80
2,2021/12/1,0:45,40374.74,350.80
3,2021/12/1,1:00,40111.55,350.80
4,2021/12/1,1:15,40067.50,348.93


In [5]:
unit.head()

,unit ID,Capacity（MW）,utilization hour (h),coal consumption (g coal/KWh),power consumption rate (%)
0,1,110.0,2069.12,266.07,6.91
1,2,160.0,5509.22,292.70,6.91
2,3,160.0,3562.79,293.35,6.91
3,4,160.0,5684.12,284.88,6.91
4,5,220.0,2231.35,323.08,8.54


`electricity_price`中，每15分钟有一个元组，包含了`(demand, price)`，即（**需求**，出清价格）二元组，比price低的能成交

`unnit`：机组是**供给者**，`utilization hour`一样的是属于一个厂子的机组，代表运行时间；`capacity`越高，发电能力越强；`coal consumption`为每发一度电的耗煤量，是成本；`power consumption`是 耗电/发电 的比率。

### 3.1 需要提交的数据

我们最后提交一个`my_submit_1.csv`的文件，按照官方:

```
初赛选手提交的预测结果文件submit.csv/submit.xlsx需要遵循下述格式。
第一行为，day,time,clearing price (CNY/MWh)；
第二行开始，每行输出day,time,clearing price (CNY/MWh)；
clearing price (CNY/MWh)的预测值应以浮点数的字符串表示形式提供，保留4位小数。
```

所以我们需要修改`electricity price`，copy一份作为提交文件

In [8]:
# 这里，我们首先找到原表中的交易价为空值的元组，再去掉原表中的demand列，这样就符合要求了
my_submit_1 = electricity_price[electricity_price["clearing price (CNY/MWh)"].isna()].drop(columns="demand")

# 此时 my_submit_1 的数据类型是 pandas.DataFrame 我们将其转换为csv
my_submit_1.to_csv(base_dir / "my_submit_1.csv", index=False)

my_submit_1.head() # 来看下处理结果

,day,time,clearing price (CNY/MWh)
55392,2023/7/1,0:15,NaN
55393,2023/7/1,0:30,NaN
55394,2023/7/1,0:45,NaN
55395,2023/7/1,1:00,NaN
55396,2023/7/1,1:15,NaN


我们需要做的，就是填满 clearing price这一列，使用AI技术来解决

此外，我们发现 `electricity price` 的前两列表示时间的列可以合一，所以我们作如下处理：

In [12]:
# 我们在原表中，新建一列 timestamp 列，由原 day + time 的拼接组成
# 此外，我们把 24 点全部替换为 第二天的 0 点。因为pandas不能处理 24 点，
# 并且我们作为敲代码的，应该有从0开始的意识，原数据从24.00开始、后面接着0.15就是应该被处理的。
electricity_price["timestamp"] = pd.to_datetime(
    electricity_price["day"] + " " + 
    electricity_price["time"].str.replace("24:00:00", "00:00"))

# 建立一个 bool 索引数组，标记所有刚刚更改的元组，为true就需要更改
mask = electricity_price['timestamp'].dt.time == pd.Timestamp('00:00:00').time()

# 将被标记的元组日期++
electricity_price.loc[mask, 'timestamp'] += pd.Timedelta(days=1)

# 设置列的顺序，同时去除day和time列
electricity_price = electricity_price[["timestamp", "demand", "clearing price (CNY/MWh)"]]

# 看看处理结果
electricity_price.head()

,timestamp,demand,clearing price (CNY/MWh)
0,2021-12-01 00:15:00,40334.18,350.80
1,2021-12-01 00:30:00,40523.15,350.80
2,2021-12-01 00:45:00,40374.74,350.80
3,2021-12-01 01:00:00,40111.55,350.80
4,2021-12-01 01:15:00,40067.50,348.93


## 4 正式开始做题

### 4.1 目标

填满 `my_submit_1.csv` 的价格列。这是个预测，根据已知条件来预测未来值，可以参见经典金融计量的方法。

我们这次按要求，需要通过**ABM模型建模**来预测市场出清价格，评价标准是 MSE 和 RMSE

### 4.2 ABM

Agent-Based-Modeling，基于个体的建模

10min 视频讲解：https://www.bilibili.com/video/BV1rU4y1x7sf/?spm_id_from=333.337.search-card.all.click&vd_source=b6823bc44ae781b7c43717114fe04aad

即：模拟/仿真 世界的模型，我们是上帝，规定好了规则，让这个世界中的各个agent（独立个体）依照规则行事，最后形成一个宏观世界

ps：这不就是世界盒子吗？
【每百年就会出现诡异灾祸！修仙世界会如何发展】 https://www.bilibili.com/video/BV1H3411g7U3/?share_source=copy_web&vd_source=841f0dab8930dfb1f8af9163985f9228

我们可以研究：在环境/规则的影响下，个体的行为能对宏观世界产生什么样的作用

**个体、环境、规则**交互作用，1+1 > 2

#### 举个例子 —— 病毒传播模型

规则如下（瞎编的）
- 这里是有1000人居住的小镇
- 初始 2 人携带病毒
- 每个人平均每天会与2个人接触
- 小镇有2个学校、5个餐厅、10个办公楼、等等设施
- 感染率 3%
- 产生抗体率 50%
- ......

模拟出来如下的模型：

![model](./model.gif)

#### 本题

- 个体：unit表中的各个机子
- 规则：机子要卖出电力，根据市场信息进行报价，并以最大利润为目的

### 4.3 经济学背景

本题的出清价格 ~ A股的集合竞价，即卖者（unit表）价低先成交，到满足demand为止。但是价格不能一味的低，我们需要有利润赚，所以价格应该“不那么低”

此外需要注意，报价有政府的上下限控制。
- 本题中：
    - 报价上限为：1300元/MWH
    - 报价下限为：-80元/MWH
    - 出清价格上限：1500元/MWH
    - 出清价格下限：-100元/MWH

通过观察数据我们也能发现，数据中有4512个-80元的交易点，最低价格为-85元。而最高价格是1296元，接近价格上限。

Task 1 中，我们使用边际成本定价法进行定价，即仅考虑生产一度电的成本，而不考虑其他。

In [13]:
# 先对成本进行升序排序，这样 unit 就以定价从低到高排序了，符合集合竞价的规则
sorted_unit = unit.sort_values("coal consumption (g coal/KWh)")  # 一度电的耗煤量，近似为边际成本
sorted_unit.head()

,unit ID,Capacity（MW）,utilization hour (h),coal consumption (g coal/KWh),power consumption rate (%)
114,115,60.0,3318.0,63.0,0.00
238,239,25.0,6486.0,73.0,5.33
148,149,30.0,3531.0,74.0,3.61
149,150,30.0,3531.0,74.0,8.72
150,151,30.0,3531.0,74.0,4.07


In [14]:
# 预先计算 sorted_unit 的累积和，方便我们知道什么时候会达到demand，从而出清
sorted_unit['cumulative_capacity'] = sorted_unit['Capacity（MW）'].cumsum()
sorted_unit.head()

,unit ID,Capacity（MW）,utilization hour (h),coal consumption (g coal/KWh),power consumption rate (%),cumulative_capacity
114,115,60.0,3318.0,63.0,0.00,60.0
238,239,25.0,6486.0,73.0,5.33,85.0
148,149,30.0,3531.0,74.0,3.61,115.0
149,150,30.0,3531.0,74.0,8.72,145.0
150,151,30.0,3531.0,74.0,4.07,175.0


In [15]:
prices = []

# 找到最后一个满足总需求的机组报价
for demand in electricity_price["demand"]:
    # 选择了cumulative_capacity列中所有大于或等于当前demand值的行。然后，从这些筛选出的行中选择coal列的第一个元素。由于我们排序过，所以价格是最小的
    price = sorted_unit[sorted_unit['cumulative_capacity'] >= demand]["coal consumption (g coal/KWh)"].iloc[0]
    prices.append(price)

print(len(prices))
prices[:5]

# 这个for循环跑了18秒。。

83520


[279.0, 279.0, 279.0, 279.0, 279.0]

In [16]:
# 拟合出 f(成本) = 定价 的函数
model = LinearRegression() # 线性拟合
# 55392为训练集的长度（即原表中价格非空的个数）
# 我们现在已知55392组数据，55392个机子用了各自的成本，形成了价格
train_length = 55392
prices = np.array(prices).reshape(-1, 1) # 转成二维数组
# 例如： [1,2,3] => [[1],[2],[3]]
X = prices[:train_length]
y = electricity_price["clearing price (CNY/MWh)"].iloc[:train_length].values.reshape(-1, 1)
model.fit(X, y)

LinearRegression()

In [18]:
print("拟合模型为：价格 = " + str(model.intercept_) + " + 成本 * " + str(model.coef_))

拟合模型为：价格 = [-2763.16464251] + 成本 * [[11.26542666]]


In [19]:
# 开始预测（历史模拟法）
y_pred = model.predict(prices[train_length:])
y_pred = y_pred.flatten()  # 2维矩阵转为1维，即把 [[1],[2],[3]] 转换为 [1,2,3]
y_pred[:5]

array([407.60234502, 401.74432316, 401.74432316, 391.1548221 ,
       401.74432316])

In [20]:
# 结果写入
my_submit_1["clearing price (CNY/MWh)"] = y_pred
my_submit_1.head()

,day,time,clearing price (CNY/MWh)
55392,2023/7/1,0:15,407.602345
55393,2023/7/1,0:30,401.744323
55394,2023/7/1,0:45,401.744323
55395,2023/7/1,1:00,391.154822
55396,2023/7/1,1:15,401.744323


In [21]:
my_submit_1.to_csv("submit.csv", index=False)